# Reference
* [`【手把手教你】入门量化回测最强神器backtrader（一）`](https://zhuanlan.zhihu.com/p/122183963)

* [`Backtrader for Backtesting (Python) – A Complete Guide`](https://algotrading101.com/learn/backtrader-for-backtesting/)

In [1]:
# !pip install autopep8
# !conda info
import platform
platform.python_version()

'3.10.8'

In [2]:
%load_ext autoreload
%autoreload 2

# 【手把手教你】入门量化回测最强神器backtrader（一）

## 引言
目前基于Python的量化回测框架有很多，开源框架有zipline、vnpy、pyalgotrader和backtrader等，而量化平台有Quantopian（国外）、聚宽、万矿、优矿、米筐、掘金等，这些量化框架或平台各有优劣。就个人而言，比较偏好用backtrader，因为它功能十分完善，有完整的使用文档，安装相对简单（直接pip安装即可）。优点是运行速度快，支持pandas的矢量运算；支持参数自动寻优运算，内置了talib股票分析技术指标库；支持多品种、多策略、多周期的回测和交易；支持pyflio、empyrica分析模块库、alphalens多因子分析模块库等；扩展灵活，可以集成TensorFlow、PyTorch和Keras等机器学习、神经网络分析模块。而不足之处在于，backtrader学习起来相对复杂，编程过程中使用了大量的元编程（类class），如果Python编程基础不扎实（尤其是类的操作），学起来会感到吃力。本文作为backtrader的入门系列之一，对其运行框架进行简要介绍，并以实际案例展示量化回测的过程。

## backtrader简介
如果将backtrader包分解为核心组件，主要包括以下组成部分：
* （1）数据加载（Data Feed）：将交易策略的数据加载到回测框架中。
* （2）交易策略（Strategy）：该模块是编程过程中最复杂的部分，需要设计交易决策，得出买入/卖出信号。
* （3）回测框架设置（ Cerebro）：需要设置（i）初始资金（ii）佣金（iii）数据馈送（iv）交易策略（v）交易头寸大小。
* （4）运行回测：运行Cerebro回测并打印出所有已执行的交易。
* （5）评估性能（Analyzers）:以图形和风险收益等指标对交易策略的回测结果进行评价。

“Lines”是backtrader回测的数据，由一系列的点组成，通常包括以下类别的数据：Open（开盘价）, High（最高价）, Low（最低价）, Close（收盘价）, Volume（成交量）, OpenInterest（无的话设置为0）。Data Feeds（数据加载）、Indicators（技术指标）和Strategies（策略）都会生成 Lines。价格数据中的所有”Open” (开盘价)按时间组成一条 Line。所以，一组含有以上6个类别的价格数据，共有6条 Lines。如果算上“DateTime”（时间，可以看作是一组数据的主键），一共有7条 Lines。当访问一条 Line 的数据时，会默认指向下标为 0 的数据。最后一个数据通过下标 -1 来访问，在-1之后是索引0，用于访问当前时刻。因此，在回测过程中，无需知道已经处理了多少条/分钟/天/月，”0”一直指向当前值，下标 -1 来访问最后一个值。

![](https://pic2.zhimg.com/v2-b722f3bcc2663b3521530a22b9bd5fd9_r.jpg)

## 回测应用实例
量化回测说白了是使用历史数据去验证交易策略的性能，因此回测的第一步是搭建交易策略，这也是backtrader要设置的最重要和复杂的部分，策略设定好后，其余部分的代码编写是手到擒来。

### 01构建策略（Strategy）
交易策略类代码包含重要的参数和用于执行策略的功能，要定义的参数或函数名如下：
* （1）params-全局参数，可选：更改交易策略中变量/参数的值，可用于参数调优。

* （2）log：日志，可选：记录策略的执行日志，可以打印出该函数提供的日期时间和txt变量。

* （3） __init__：用于初始化交易策略的类实例的代码。

* （4）notify_order，可选：跟踪交易指令（order）的状态。order具有提交，接受，买入/卖出执行和价格，已取消/拒绝等状态。

* （5）notify_trade，可选：跟踪交易的状态，任何已平仓的交易都将报告毛利和净利润。

* （6）next，必选：制定交易策略的函数，策略模块最核心的部分。

下面以一个简单的单均线策略为例，展示backtrader的使用过程，即**当收盘价上涨突破20日均线买入（做多），当收盘价下跌跌穿20日均线卖出（做空）**。为简单起见，不报告交易回测的日志，因此log、notify_order和notify_trade函数省略不写。

# Backtrader for Backtesting (Python) – A Complete Guide

## Why should I learn Backtrader?
* `Backtesting` - This might seem like an obvious one but Backtrader removes the tedious process of cleaning up your data and iterating through it to test strategies. It has built-in templates to use for various data sources to make importing data easier.
* `Optimizing` - Adjusting a few parameters can sometimes be the difference between a profitable strategy and an unprofitable one. After running a backtest, optimizing is easily done by changing a few lines of code.
* `Plotting` – If you’ve worked with a few Python plotting libraries, you’ll know these are not always easy to configure, especially the first time around. A complex chart can be created with a single line of code.
* `Indicators` – Most of the popular indicators are already programmed in the Backtrader platform. This is especially useful if you want to test out an indicator but you’re not sure how effective it will be. Rather than trying to figure out the math behind the indicator, and how to code it, you can test it out first in Backtrader, probably with one line of code.
* `Support for Complex Strategies` – Want to take a signal from one dataset and execute a trade on another? Does your strategy involve multiple timeframes? Or do you need to resample data? Backtrader has accounted for the various ways traders approach the markets and has extensive support.
* `Open Source`
* `Active Development`– This might be one area where Backtrader especially stands out. The framework was originally developed in 2015 and constant improvements have been made since then. Just a few weeks ago, a pandas-based technical analysis library was released to address issues in the popular and commonly used TA-Lib framework. Further, with a wide user base, there is also active third-party development.
* `Live Trading` – If you’re happy with your backtesting results, it is easy to migrate to a live environment within Backtrader. This is especially useful if you plan to use the built-in indicators offered by the platform.

## Choosing which IDE to use with Backtrader
While it is possible to use interactive IDE’s for some functionality in Backtrader, it is not recommended. There are certain functions, such as optimization, that require multiprocessing which does not work well with interactive IDE’s.

In [53]:
import os
import numpy as np
import pandas as pd
import yfinance as yf
import backtrader as bt
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

# plt.style.use('seaborn')

In [54]:
print(matplotlib.__version__)
print(np.__version__)
print(pd.__version__)
print(bt.__version__)

3.7.1
1.24.2
2.0.0
1.9.76.123


## How to configure the basic Backtrader setup
There are two main components to setting up your basic Backtrader script. 
* The strategy class
* the cerebro engine

In [55]:
class MyStrategy(bt.Strategy):
    def next(self):
        pass #Do something

#Instantiate Cerebro engine
cerebro = bt.Cerebro()

#Add strategy to Cerebro
cerebro.addstrategy(MyStrategy)

#Run Cerebro Engine
cerebro.run()

[]

We will go into the strategy class in more detail in the examples that follow. This is where all the logic goes in determining and executing your trade signals. It is also where indicators can be created or called, and where you can determine what get’s logged or printed to screen.

The cerebro engine is the core of Backtrader. This is the main class and we will add our data and strategies to it before eventually calling the `cerebro.run()` command.

## How to get data and import it into Backtrader

In [31]:
TSLA = yf.Ticker(
    "TSLA",
#     start='2020-01-01',
#     end='2023-03-26',
)

In [32]:
hist_tsla = TSLA.history(period="5y")

In [33]:
hist_tsla.to_csv(
    os.path.join('data', 'tsla_5y.csv')
)

In [46]:
hist_tsla.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2018-04-09 00:00:00-04:00,20.024668,20.633333,19.280666,19.310667,153747000,0.0,0.0
2018-04-10 00:00:00-04:00,19.931334,20.473333,19.578667,20.313334,164847000,0.0,0.0
2018-04-11 00:00:00-04:00,20.049334,20.598667,19.977333,20.062000,112243500,0.0,0.0
2018-04-12 00:00:00-04:00,20.154667,20.263332,19.578667,19.605333,114132000,0.0,0.0
2018-04-13 00:00:00-04:00,20.240000,20.263332,19.732000,20.022667,109908000,0.0,0.0


In [52]:
# hist_tsla.index

In [57]:
data = bt.feeds.YahooFinanceCSVData(
#     dataname=os.path.join('data', 'tsla_5y.csv')
    dataname=os.path.join('data', 'TLSA.csv')
)

In [59]:
cerebro.adddata(data)

In [61]:
# help(data)

## How to print or log data using the strategy class in Backtrader
To get a bit more familiar with the Strategy class in Backtrader, we will create a simple script that prints the closing prices for our dataset. The Strategy class is where we will be spending most of our time within Backtrader.

The first thing we will do is create a new class called `PrintClose` which inherits the Backtrader Strategy class.

In [62]:
import backtrader as bt


class PrintClose(bt.Strategy):

    def __init__(self):
        print(self.datas[0].close)
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()} {txt}')  # Print date and close

    def next(self):
        self.log('Close: ', self.dataclose[0])


# Instantiate Cerebro engine
cerebro = bt.Cerebro()

# Add data feed to Cerebro
data = bt.feeds.YahooFinanceCSVData(
    dataname=os.path.join('data', 'tsla_5y.csv'))
cerebro.adddata(data)

# Add strategy to Cerebro
cerebro.addstrategy(PrintClose)

# Run Cerebro Engine
cerebro.run()

AttributeError: 'float' object has no attribute 'isoformat'